#### Convolutional Neural Networks for Sentence Classification

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [5]:
# Baixando e carregando o dataset AG_NEWS
train_iter = AG_NEWS(split='train')

# Criando um tokenizador
tokenizer = get_tokenizer('basic_english')

# Construindo o vocabulário a partir dos dados de treinamento
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

NameError: name 'portalocker' is not defined
This exception is thrown by __iter__ of _MemoryCellIterDataPipe(remember_elements=1000, source_datapipe=_ChildDataPipe)

In [ ]:
# Definindo o modelo da CNN
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_filters, kernel_sizes, hidden_dim, num_classes):
        super(TextCNN, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (k, embed_dim)) for k in kernel_sizes
        ])
        self.fc = nn.Linear(len(kernel_sizes) * num_filters, hidden_dim)
        self.out = nn.Linear(hidden_dim, num_classes)

    def forward(self, text):
        embedded = self.embedding(text)
        
        # .unsqueeze(1) serve para criar uma dimensão a mais na posição passada por parametros.
        # É principalmente usado para concatenar tensores que representam palavras.
        embedded = embedded.unsqueeze(1)
        conv_outputs = []
        for conv in self.convs:
            conv_output = F.relu(conv(embedded))
            pool_output = F.max_pool2d(conv_output, (conv_output.shape[2], 1))
            conv_outputs.append(pool_output.squeeze(3))
        cat_outputs = torch.cat(conv_outputs, 1)
        fc_output = self.fc(cat_outputs)
        fc_output = F.relu(fc_output)
        out = self.out(fc_output)
        return out

In [ ]:
# Definindo hiperparâmetros do modelo
VOCAB_SIZE = len(vocab)
EMBED_DIM = 32
NUM_FILTERS = 100
KERNEL_SIZES = [3, 4, 5]
HIDDEN_DIM = 64
NUM_CLASSES = 4

# Instanciando o modelo
model = TextCNN(VOCAB_SIZE, EMBED_DIM, NUM_FILTERS, KERNEL_SIZES, HIDDEN_DIM, NUM_CLASSES)

# Definindo a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
# Treinando o modelo
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        text, label = batch.text, batch.label
        predictions = model(text).squeeze(1)
        loss = criterion(predictions, label)
        acc = accuracy(predictions, label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# Avaliando a acurácia do modelo
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text, label = batch.text